### 사전준비
- sqlalchemy 설치

In [ ]:
%pip install SQLAlchemy

### sqlalchemy 

- SQLAlchemy is an open-source SQL toolkit and object-relational mapper (ORM) for the Python programming language released under the MIT License.

- sqlalchemy 구성

![Alt text for broken image link](../resources/sqlalchemy.jpg)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///mydb.db')

In [ ]:
conn = engine.connect()

# db file 생성 확인 

### SQL
-  sql 문장 실행하기

In [ ]:
from sqlalchemy import text
conn.execute(text("CREATE TABLE TEST (X int, Y int)")) 

In [ ]:
insert_statement = "INSERT INTO TEST (x, y) VALUES (:x, :y)"
test_data = [{"x": 1, "y": 1}, {"x": 2, "y": 4}]

conn.execute(text(insert_statement), test_data)
conn.commit()

In [ ]:
select_statement = "SELECT x, y FROM TEST"
result = conn.execute(text(select_statement))
for row in result:
    print(row)

In [ ]:
conn.execute(text("DROP TABLE TEST")) 

### ORM

In [ ]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

- Model(DB Table) 정의

In [ ]:
"""
prompt :

sqlalchemy.orm.declarative_base 의 결과를 상속 받은 data model 두개를 아래 조건에 맞게 선언하는 code를 작성하라
---
1. 
class 이름 : User
table 이름 : user_account
칼럼들 :
    name (type:string)
    fullname  (type:string)
2. 
class 이름 : Asset
table 이름 : user_asset
칼럼들 :
    asset_name (type:string)
    owner_id  (type:integer, user_account.id를 참조하는 foreign key)

기타조건 : 
- user_account table 한 row 가 삭제되면 user_asset.owner_id로 연결된 row들도 삭제
- table 객체를 print하면 table의 column과 값을 출력한다.
===
results from ChatGPT-3.5
"""
sqlalchemy.orm.declarative_base 

from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = 'user_account'  # 데이터베이스에서 사용할 테이블 이름입니다.

    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)

    # User와 Asset의 관계를 설정합니다.
    assets = relationship("Asset", cascade="all, delete-orphan")

    # 객체를 print 하면 호출됨
    def __repr__(self):
        return f"User(id={self.id}, name={self.name}, fullname={self.fullname})"
    
class Asset(Base):
    __tablename__ = 'user_asset'
    id = Column(Integer, primary_key=True)
    asset_name = Column(String(30))
    
    # User 테이블의 id를 외래키로 설정하여 User와의 관계를 맺습니다.
    owner_id = Column(Integer, ForeignKey('user_account.id'))

    # 객체를 print 하면 호출됨
    def __repr__(self):
        return f"Asset(id={self.id}, asset_name={self.asset_name}, owner_id={self.owner_id})"


In [ ]:
print('tables : ', Base.metadata.tables)

- Database에 Table 일괄 생성

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
#Base.metadata.drop_all(engine)

- sqlalchemy로부터 db session을 하나 얻음

In [ ]:
from sqlalchemy.orm import Session

session = Session(engine)

- Insert

In [ ]:
me = User(name='Kim', fullname='Hennry Kim')

session.add(me)

In [ ]:
session.commit()

In [ ]:
my_son = User(name='Kim', fullname='Sun Kim')

session.add(my_son)

session.rollback()

- Select

In [ ]:
users = session.query(User).all()

for user in users:
    print(user)

In [ ]:
user = session.query(User).first()

print(user)

In [ ]:
user = session.query(User).filter(User.name=='Kim')

for user in users:
    print(user)

- Update

In [ ]:
user.fullname = '김형기'
print('Before commit : Is the session dirty?',user in session.dirty)
session.commit()
print('After commit : Is the session dirty?',user in session.dirty)

- Delete

In [ ]:
session.delete(user)
session.commit()

- Child Table Insert

In [ ]:
me = User(name='Kim', fullname='Hennry Kim')
me.assets.append(Asset(asset_name="Car"))
me.assets.append(Asset(asset_name="House"))
session.add(me)
session.commit()

In [ ]:
me.assets.append(Asset(asset_name="Wife"))
me.assets.append(Asset(asset_name="Son1"))
me.assets.append(Asset(asset_name="Son2"))
session.commit()

In [ ]:
for asset in me.assets:
    print(asset)

- Child Table로부터 Select

-
    - Asset Table을 직접 Select

In [ ]:
assets = session.query(Asset).filter(Asset.owner_id==me.id, Asset.asset_name=='Car')

for asset in assets:
    print(asset)

In [154]:
rows = session.query(User, Asset).join(Asset, Asset.owner_id==User.id).filter(User.id==me.id)

for row in rows:
    print(row[0], row[1])

User(id=2, name=nobody, fullname=nobody) Asset(id=1, asset_name=no pain)
User(id=2, name=nobody, fullname=nobody) Asset(id=2, asset_name=no gain)


-
    - Comprehention을 사용해 me 객체에서 Select

In [ ]:
# asset_name으로 검색이 가능하도록 key=asset_name, value=Asset instance인 dictionary 생성
assets_in_me = { asset.asset_name:asset for asset in me.assets }

print('Assets : ', assets_in_me)
print('asset_name이 Car인 Asset  : ', assets_in_me['Car'])
print('asset_name이 Car인 Asset의 id  : ', assets_in_me['Car'].id)

In [ ]:
asset = next( a for a in me.assets if a.asset_name=="Car")

print('asset_name이 Car인 Asset  : ', asset)
print('asset_name이 Car인 Asset의 id  : ', asset.id)

- Child Table Update

In [ ]:
asset.asset_name = "비싼 Car"

- Child Table Delete

In [ ]:
user.assets.remove(asset)

In [ ]:
session.rollback()

In [ ]:
me = User(name='Kim', fullname='Hennry Kim')

session.add(me)

### Core

- Insert

In [ ]:
from sqlalchemy import insert

stmt = insert(User).values(name='JK', fullname="JK Choi")

session.execute(stmt)
session.commit()

In [ ]:
print(stmt)

- Select

In [ ]:
from sqlalchemy import select

stmt = select(User).where(User.name == 'JK')

cursor = session.execute(stmt)

for row in cursor:
    print(row)


In [ ]:
print(stmt)

- Update

In [ ]:
from sqlalchemy import update

stmt = update(User).where(User.name == 'JK').\
    values(fullname='진국')

session.execute(stmt)
session.commit()

In [ ]:
print(stmt)

- Delete

In [ ]:
from sqlalchemy import delete

stmt = delete(User).where(User.name == 'JK')

session.execute(stmt)
session.commit()

In [ ]:
print(stmt)

- Child Table Insert

In [142]:
# Insert User
user_insert = insert(User).values(name='nobody', fullname='nobody')
result = session.execute(user_insert)

user_id = result.lastrowid

# Insert Asset
asset1_insert = insert(Asset).values(asset_name="no pain", owner_id=user_id)
asset2_insert = insert(Asset).values(asset_name="no gain", owner_id=user_id)

asset1_id = session.execute(asset1_insert).lastrowid
asset2_id = session.execute(asset2_insert).lastrowid

session.commit()

- Select join tables

In [149]:
stmt = select(User, Asset).where(User.id == user_id, User.id == Asset.owner_id)

rows = session.execute(stmt)

# 결과 출력
for row in rows:
    print("User ID:", row[0].id, "Name:", row[0].name, "Asset ID:", row[1].id, "Asset Name:", row[1].asset_name)

User ID: 2 Name: nobody Asset ID: 1 Asset Name: no pain
User ID: 2 Name: nobody Asset ID: 2 Asset Name: no gain
